In [1]:
import os

In [2]:
pwd

'c:\\Prajesh\\personal\\ML_learning\\my_work\\kubeflow_app\\kubeflow_ML_app\\ML_flow_app\\research'

In [3]:
cd ..

c:\Prajesh\personal\ML_learning\my_work\kubeflow_app\kubeflow_ML_app\ML_flow_app


In [4]:
os.environ["MLFLOW_TRACKING_URI"]='http://ec2-15-206-173-174.ap-south-1.compute.amazonaws.com:5000/'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelEvaluationConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: str
    target_col: str
    mlflow_uri: str

In [6]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self):
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN
        params = self.params.RandomForestRegressor
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_data_path,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_col=schema.name,
            mlflow_uri = config.mlflow_uri
        )
        
        return model_evaluation_config

In [8]:
# dummy = ConfigurationManager()
# dummy.get_model_evaluation_config()

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score
import os
import pandas as pd
import mlflow
import mlflow.sklearn
from urllib.parse import urlparse
import numpy as np
import joblib
from mlFlowProject.utils.common import save_json
from pathlib import Path

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        
    def evaluate_model(self, ytrue, ypred):
        rmse = np.sqrt(mean_squared_error(ytrue, ypred))
        mae = mean_absolute_error(ytrue, ypred)
        r2 = r2_score(ytrue, ypred)
        
        train_data = pd.read_csv(self.config.train_data_path)
        xtrain = np.array(train_data['Open']).reshape(-1, 1)
        ytrain = np.array(train_data[[self.config.target_col]]).reshape(-1)
        
        model = joblib.load(self.config.model_path)
        
        cv_score = cross_val_score(model, xtrain, ytrain, cv = 5)
        return rmse, mae, r2, cv_score
    
    def log_into_mlflow(self):
        
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        
        xtest = np.array(test_data['Open']).reshape(-1, 1)
        ytest = np.array(test_data[[self.config.target_col]]).reshape(-1)
        
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            
            ypred = model.predict(xtest)
            
            (rmse, mae, r2, cv_score) = self.evaluate_model(ytest, ypred)
            
            cv_score = cv_score.tolist()
            
            scores = {
                "rmse": rmse,
                "mae": mae,
                "r2": r2,
                "mean_cv_accuracy": np.mean(cv_score),
                "cv_accuracy_std": np.std(cv_score)
            }
            
            save_json(path = Path(self.config.metric_file_name), data = scores)
            
            mlflow.log_params(self.config.all_params)
            
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("mae", mae)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mean_cv_accuracy", np.mean(cv_score))
            mlflow.log_metric("cv_accuracy_std", np.std(cv_score))
            
            # if tracking_url_type_store != "file":
            #     mlflow.sklearn.log_model(model, "model", registered_model_name = "RandomForestRegressor")
            # else:
            mlflow.sklearn.log_model(model, "model")
        
        

In [11]:
try:
    config_manager = ConfigurationManager()
    model_eval_config = config_manager.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config = model_eval_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    raise e

[2023-08-07 15:09:23,373: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-07 15:09:23,375: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-07 15:09:23,377: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-07 15:09:23,379: INFO: common: created directory at: artifacts]
[2023-08-07 15:09:23,381: INFO: common: created directory at: artifacts/model_evaluation]
[2023-08-07 15:09:23,538: DEBUG: cmd: Popen(['git', 'version'], cwd=c:\Prajesh\personal\ML_learning\my_work\kubeflow_app\kubeflow_ML_app\ML_flow_app, universal_newlines=False, shell=None, istream=None)]
[2023-08-07 15:09:23,604: DEBUG: cmd: Popen(['git', 'version'], cwd=c:\Prajesh\personal\ML_learning\my_work\kubeflow_app\kubeflow_ML_app\ML_flow_app, universal_newlines=False, shell=None, istream=None)]
[2023-08-07 15:09:23,729: DEBUG: util: Failed checking if running in CYGWIN due to: FileNotFoundError(2, 'The system cannot find the file specified', None, 2, None)]
[2023-

c:\Users\Natwar\AppData\Local\Programs\Python\Python37\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


[2023-08-07 15:09:29,421: DEBUG: hooks: Changing event name from creating-client-class.iot-data to creating-client-class.iot-data-plane]
[2023-08-07 15:09:29,424: DEBUG: hooks: Changing event name from before-call.apigateway to before-call.api-gateway]
[2023-08-07 15:09:29,426: DEBUG: hooks: Changing event name from request-created.machinelearning.Predict to request-created.machine-learning.Predict]
[2023-08-07 15:09:29,428: DEBUG: hooks: Changing event name from before-parameter-build.autoscaling.CreateLaunchConfiguration to before-parameter-build.auto-scaling.CreateLaunchConfiguration]
[2023-08-07 15:09:29,430: DEBUG: hooks: Changing event name from before-parameter-build.route53 to before-parameter-build.route-53]
[2023-08-07 15:09:29,431: DEBUG: hooks: Changing event name from request-created.cloudsearchdomain.Search to request-created.cloudsearch-domain.Search]
[2023-08-07 15:09:29,432: DEBUG: hooks: Changing event name from docs.*.autoscaling.CreateLaunchConfiguration.complete-se

c:\Users\Natwar\AppData\Local\Programs\Python\Python37\lib\site-packages\boto3\compat.py:82: PythonDeprecationWarning: Boto3 will no longer support Python 3.7 starting December 13, 2023. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.8 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


[2023-08-07 15:09:29,884: DEBUG: loaders: Loading JSON file: c:\Users\Natwar\AppData\Local\Programs\Python\Python37\lib\site-packages\botocore\data\s3\2006-03-01\service-2.json]
[2023-08-07 15:09:30,105: DEBUG: loaders: Loading JSON file: c:\Users\Natwar\AppData\Local\Programs\Python\Python37\lib\site-packages\botocore\data\s3\2006-03-01\endpoint-rule-set-1.json.gz]
[2023-08-07 15:09:30,127: DEBUG: loaders: Loading JSON file: c:\Users\Natwar\AppData\Local\Programs\Python\Python37\lib\site-packages\botocore\data\partitions.json]
[2023-08-07 15:09:30,131: DEBUG: hooks: Event creating-client-class.s3: calling handler <function add_generate_presigned_post at 0x000002419707A798>]
[2023-08-07 15:09:30,132: DEBUG: hooks: Event creating-client-class.s3: calling handler <function lazy_call.<locals>._handler at 0x0000024196D460D8>]
[2023-08-07 15:09:30,245: DEBUG: hooks: Event creating-client-class.s3: calling handler <function add_generate_presigned_url at 0x000002419707A558>]
[2023-08-07 15:09